In [ ]:
import pandas as pd
import scipy.optimize as opt
import regularized_optimization as reg_opt
import Utilityfunctions as utils
import jax.numpy as jnp
import numpy as np
import jax as jax
import matplotlib.pyplot as plt
import simulations as simul
jax.config.update("jax_enable_x64", True)

In [ ]:
#params = pd.read_csv("../results/paad_panet_" + "muts_30_5_08_008" + ".csv")
params = pd.read_csv("../results/paad_mixed_08_003.csv")
th_learned = params.iloc[:,1:-1].to_numpy()
lam1 = np.exp(params.iloc[0,-1])
lam2 = np.exp(params.iloc[1, -1])
events = params.columns[1:-1]
n_sim = 100000
n = th_learned.shape[0]
rng = np.random.default_rng(42)


In [ ]:
probs_prim, probs_met = simul.p_full_orders(th_learned, n_sim, lam1, lam2, rng)
cum_probs_prim = np.cumsum(probs_prim, axis=0)
cum_probs_met = np.cumsum(probs_met, axis=0)

In [ ]:
print(probs_prim)

In [ ]:
fig,ax = plt.subplots(ncols=2, nrows=n,figsize=(16,150))
for i in range(n):
    ax[i, 0].bar(np.arange(1,n+1), probs_prim[:,i], alpha=0.4, label="PT")
    ax[i, 0].bar(np.arange(1,n+1), probs_met[:,i], color="red", alpha=0.4, label="MT")
    ax[i, 1].plot(np.arange(1,n+1), cum_probs_prim[:,i],alpha=0.4, label="PT" )
    ax[i, 1].plot(np.arange(1,n+1), cum_probs_met[:,i], color="red", alpha=0.4, label="MT" )
    ax[i, 0].set_title(events[i]+" Order of accumulation")
    ax[i, 1].set_title(events[i]+" Cummulative probability")
    ax[i,0].legend()
    ax[i,1].legend()

In [ ]:
probs_pre_seed = simul.p_mut_pre_seed(th_learned, n_sim, lam1, lam2, rng)
iwas = pd.DataFrame(probs_pre_seed[:-1], index=events[:-1], columns=["probs"])
iwas2 = iwas.sort_values(by=["probs"])

In [ ]:
fig, ax = plt.subplots()
plt.style.use("default")
ax.set_ylim(0,1)
ax.axhline(0.5, color ="r", linestyle="dashed")
iwas2.plot(kind="bar", ax=ax, alpha=0.4)